# The Optimist Experiment



In [ ]:

class Hypothesis:
    """
    A base class for graph hypotheses.
    """
    def __init__(self, statement, true_object_set=None):
        self.statement = statement
        self.true_object_set = true_object_set

    def __str__(self):
        return f"{self.statement}"

    def __repr__(self):
        return f"{self.statement}"

    def __call__(self, name, df):
        return df.loc[df["name"] == f"{name}.txt"][self.statement]

    def _le__(self, other):
        return len(self.true_object_set) <= len(other.true_object_set)

    def __lt__(self, other):
        return len(self.true_object_set) < len(other.true_object_set)

    def __ge__(self, other):
        return len(self.true_object_set) >= len(other.true_object_set)

    def __gt__(self, other):
        return len(self.true_object_set) > len(other.true_object_set)

    def __eq__(self, other):
        return self.__str__() == other.__str__()

    def __hash__(self):
        return hash((self.statement))

class Conclusion:
    """
    A base class for graph conclusions.
    """
    def __init__(self, lhs, inequality, rhs, intercept=0):
        self.lhs = lhs
        self.inequality = inequality
        self.rhs = rhs
        self.intercept = intercept

    def __str__(self):
        raise NotImplementedError("Subclasses must implement __str__")

    def __repr__(self):
        raise NotImplementedError("Subclasses must implement __repr__")

    def __call__(self, name, df):
        raise NotImplementedError("Subclasses must implement __call__")

    def __hash__(self):
        return hash((str(self)))

class Conjecture:
    """
    A base class for graph conjectures.
    """
    def __init__(self, hypothesis, conclusion, symbol="G", touch=0, sharps=None, difficulty=0):
        self.hypothesis = hypothesis
        self.conclusion = conclusion
        self.symbol = symbol
        self.touch = touch
        self.difficulty = difficulty

        self.sharps = set(sharps)

    def __str__(self):
        hypothesis = f"If {self.symbol} is {self.hypothesis}"
        return f"{hypothesis}, then {self.conclusion}"

    def __repr__(self):
        hypothesis = f"If {self.symbol} is {self.hypothesis}"
        return f"{hypothesis}, then {self.conclusion}"

    def __call__(self, name, df):
        if self.hypothesis(name, df).values[0]:
            return self.conclusion(name, df)
        else:
            return False

    def get_sharp_graphs(self, df):
        raise NotImplementedError("Subclasses must implement get_sharp_graphs")

    def __eq__(self, other):
        return self.__str__() == other.__str__()


    def __hash__(self):
        return hash((str(self)))


class MultiLinearConclusion(Conclusion):
    """
    A class for multilinear graph conclusions.
    """
    def __init__(self, lhs, inequality, slopes, rhs, intercept):
        super().__init__(lhs, inequality, rhs, intercept)
        self.slopes = slopes

    def __str__(self):
        slope_terms = []
        for m, rhs in zip(self.slopes, self.rhs):
            if m == 1:
                slope_terms.append(f"{rhs}")
            elif m == -1:
                slope_terms.append(f"- {rhs}")
            elif m != 0:
                slope_terms.append(f"{m} * {rhs}")

        slope_str = " + ".join(slope_terms)

        if self.intercept > 0:
            result = f"{slope_str} + {self.intercept}"
        elif self.intercept < 0:
            result = f"{slope_str} - {abs(self.intercept)}"
        else:
            result = slope_str

        result = result.replace("+ -", "-").strip()
        return f"{self.lhs} {self.inequality} {result}"

    def __repr__(self):
        return self.__str__()

    def __call__(self, name, df):
        data = df.loc[df["name"] == f"{name}"]
        rhs_value = sum(m * data[r].values[0] for m, r in zip(self.slopes, self.rhs)) + self.intercept
        if self.inequality == "<=":
            return data[self.lhs].values[0] <= rhs_value
        elif self.inequality == ">=":
            return data[self.lhs].values[0] >= rhs_value
        else:
            data[self.lhs].values[0] == rhs_value

    def __eq__(self, other):
        return self.__str__() == other.__str__()

    def reversal(self):
        if self.inequality == "<=":
            return MultiLinearConclusion(self.lhs, ">=", self.slopes, self.rhs, self.intercept)
        elif self.inequality == ">=":
            return MultiLinearConclusion(self.lhs, "<=", self.slopes, self.rhs, self.intercept)

    def rhs_evaluate(self, x):
        return sum(m * x for m in self.slopes) + self.intercept

    def __hash__(self):
        return hash((str(self)))


class MultiLinearConjecture(Conjecture):
    """
    A class for multilinear graph conjectures.
    """
    def __repr__(self):
        hypothesis = f"If {self.symbol} is {self.hypothesis}"
        return f"{hypothesis}, then {self.conclusion}."

    def get_sharp_graphs(self, df):
        return df.loc[(df[self.hypothesis.statement] == True) &
                      (df[self.conclusion.lhs] == sum(self.conclusion.slopes[i] * df[self.conclusion.rhs[i]]
                                                      for i in range(len(self.conclusion.rhs))) + self.conclusion.intercept)]

    def __eq__(self, other):
        return self.hypothesis == other.hypothesis and self.conclusion == other.conclusion

    def false_graphs(self, df):
        if self.conclusion.inequality == "<=":
            return df.loc[(df[self.hypothesis.statement] == True) &
                          (df[self.conclusion.lhs] > sum(self.conclusion.slopes[i] * df[self.conclusion.rhs[i]]
                                                         for i in range(len(self.conclusion.rhs))) + self.conclusion.intercept)]
        elif self.conclusion.inequality == ">=":
            return df.loc[(df[self.hypothesis.statement] == True) &
                          (df[self.conclusion.lhs] < sum(self.conclusion.slopes[i] * df[self.conclusion.rhs[i]]
                                                         for i in range(len(self.conclusion.rhs))) + self.conclusion.intercept)]
        else:
            return df.loc[(df[self.hypothesis.statement] == True) &
                          (df[self.conclusion.lhs] != sum(self.conclusion.slopes[i] * df[self.conclusion.rhs[i]]
                                                         for i in range(len(self.conclusion.rhs))) + self.conclusion.intercept)]
    def is_equal(self):
        return self.conclusion.inequality == "="

    def get_functions(self, invariant_dict):
        """
        Returns a tuple of functions (target_function, rhs_function) that compute the LHS and RHS
        of the conjecture for a given networkx graph.

        Parameters:
        - invariant_dict: A dictionary where the keys are the string names of the invariants
                          and the values are the corresponding functions that compute them.

        Returns:
        - A tuple (target_function, rhs_function)
        """

        # Define the target_function to compute the LHS invariant
        def target_function(graph):
            if self.conclusion.lhs in invariant_dict:
                return invariant_dict[self.conclusion.lhs](graph)
            else:
                raise ValueError(f"LHS invariant '{self.conclusion.lhs}' not found in the dictionary.")

        # Define the rhs_function to compute the RHS expression
        def rhs_function(graph):
            rhs_value = 0
            for slope, rhs_invariant in zip(self.conclusion.slopes, self.conclusion.rhs):
                if rhs_invariant in invariant_dict:
                    rhs_value += slope * invariant_dict[rhs_invariant](graph)
                else:
                    raise ValueError(f"RHS invariant '{rhs_invariant}' not found in the dictionary.")
            return rhs_value + self.conclusion.intercept

        def hypothesis_function(graph):
            if self.hypothesis in invariant_dict:
                return invariant_dict[self.hypothesis](graph)
            else:
                raise ValueError(f"Hypothesis invariant '{self.hypothesis}' not found in the dictionary.")

        return hypothesis_function, target_function, rhs_function

    def get_penality_function(self, penality_dict):
        """
        Returns a function that computes the penalty for a given networkx graph.
        """
        def penality_function(graph):
            hyp = str(self.hypothesis)
            if hyp in penality_dict:
                return penality_dict[hyp](graph)
            else:
                raise ValueError(f"Hypothesis '{hyp}' not found in the dictionary.")

        return penality_function

    def plot(self, df):
        import matplotlib.pyplot as plt
        import seaborn as sns
        sns.set_theme()

        if len(self.conclusion.slopes) == 1:
            # Filter dataframe where the hypothesis holds
            df = df[df[self.hypothesis.statement] == True]

            # Set up data for plotting
            y = df[self.conclusion.lhs]
            x = df[self.conclusion.rhs]
            rhs = self.conclusion.rhs_evaluate(x)

            # Create a figure and axis object
            fig, ax = plt.subplots(figsize=(10, 10))

            # Plot the data
            ax.set_title(f"{self.__repr__()}")
            ax.scatter(x, y, color='blue', label=f'Data')
            ax.plot(x, rhs, color='red', label=f'Prediction: {self.conclusion}')

            # Set labels and grid
            ax.set_xlabel(self.conclusion.rhs)
            ax.set_ylabel(self.conclusion.lhs)
            ax.grid(True)
            ax.legend()

            # Return the figure
            return fig
        else:
            print("Cannot plot multi-linear conjectures")
            return None

    def __hash__(self):
        return hash((str(self)))

    def __eq__(self, other):
        return self.__str__() == other.__str__()


In [2]:
from fractions import Fraction
from itertools import combinations
from pulp import LpProblem, LpMaximize, LpVariable, lpSum

def make_linear_conjectures(
        df,
        target,
        others,
        hyp="a connected graph",
        symbol="G",
    ):
    """
    Returns a MultiLinearConjecture object with different slopes for upper and lower bounds,
    using a mixed-integer program to maximize the number of equalities on the extreme values.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the data.
    target : string
        The name of the target variable.
    others : list of strings
        A list of the names of other variables (invariants).
    hyp : string
        The name of the hypothesis variable.
    symbol : string
        The symbol of the object in the conjecture.

    Returns
    -------
    MultiLinearConjecture
        The conjecture with both upper and lower bounds, each with different slopes.
    """

    # Filter data for the hypothesis condition.
    df = df[df[hyp] == True]
    true_objects = df["name"].tolist()

    # Preprocess the data to find the maximum Y for each X for the upper bound
    df_upper = df.loc[df.groupby(others)[target].idxmax()]
    # Preprocess the data to find the minimum Y for each X for the lower bound
    df_lower = df.loc[df.groupby(others)[target].idxmin()]

    # Extract the data for the upper and lower bound problems
    Xs_upper = [df_upper[other].tolist() for other in others]
    Y_upper = df_upper[target].tolist()
    Xs_lower = [df_lower[other].tolist() for other in others]
    Y_lower = df_lower[target].tolist()

    # Initialize the MIP problem.
    prob = LpProblem("Maximize_Equality", LpMaximize)

    # Initialize the variables for the MIP (one set for upper bound and one for lower bound).
    ws_upper = [LpVariable(f"w_upper{i+1}", upBound=4, lowBound=-4) for i in range(len(others))]  # Weights for upper bound
    ws_lower = [LpVariable(f"w_lower{i+1}", upBound=4, lowBound=-4) for i in range(len(others))]  # Weights for lower bound
    b_upper = LpVariable("b_upper", upBound=3, lowBound=-3)
    b_lower = LpVariable("b_lower", upBound=3, lowBound=-3)

    # Binary variables z_j^upper and z_j^lower to maximize equality conditions for extreme points
    z_upper = [LpVariable(f"z_upper{j}", cat="Binary") for j in range(len(Y_upper))]
    z_lower = [LpVariable(f"z_lower{j}", cat="Binary") for j in range(len(Y_lower))]

    M = 1000  # Big-M value

    # Upper bound constraints (maximize equality on max Y values)
    for j in range(len(Y_upper)):
        prob += lpSum([ws_upper[i] * Xs_upper[i][j] for i in range(len(others))]) + b_upper >= Y_upper[j]
        prob += lpSum([ws_upper[i] * Xs_upper[i][j] for i in range(len(others))]) >= b_upper
        prob += lpSum([ws_upper[i] * Xs_upper[i][j] for i in range(len(others))]) + b_upper - Y_upper[j] <= M * (1 - z_upper[j])

    # Lower bound constraints (maximize equality on min Y values)
    for j in range(len(Y_lower)):
        prob += lpSum([ws_lower[i] * Xs_lower[i][j] for i in range(len(others))]) + b_lower <= Y_lower[j]
        prob += lpSum([ws_lower[i] * Xs_lower[i][j] for i in range(len(others))]) >= b_lower
        prob += Y_lower[j] - lpSum([ws_lower[i] * Xs_lower[i][j] for i in range(len(others))]) - b_lower <= M * (1 - z_lower[j])

    # Maximize the number of equalities for both upper and lower bounds
    prob += lpSum(z_upper) + lpSum(z_lower)

    # Solve the MIP
    prob.solve()

    if prob.status != 1:
        print("No feasible solution found.")
        return None
    else:
        weights_upper = [Fraction(w.varValue).limit_denominator(10) for w in ws_upper]
        weights_lower = [Fraction(w.varValue).limit_denominator(10) for w in ws_lower]
        b_upper_value = Fraction(b_upper.varValue).limit_denominator(10)
        b_lower_value = Fraction(b_lower.varValue).limit_denominator(10)

        if weights_lower == weights_upper and b_upper_value == b_lower_value:
            touch_upper = len(true_objects)

            # Create the hypothesis and conclusion objects for both upper and lower bounds.
            hypothesis = Hypothesis(hyp, true_object_set=true_objects)
            upper_conclusion = MultiLinearConclusion(target, "=", weights_upper, others, b_upper_value)

            # Return the full conjecture object (not the conclusion directly).
            return MultiLinearConjecture(hypothesis, upper_conclusion, symbol, touch_upper, true_objects), None
        else:
            Xs_true_upper = [df[other].tolist() for other in others]
            Y_true_upper = df[target].tolist()
            Xs_true_lower = [df[other].tolist() for other in others]
            Y_true_lower = df[target].tolist()
            # Compute the number of instances of equality - the touch number of the conjecture.
            touch_set_upper = set([true_objects[j] for j in range(len(Y_true_upper)) if
                                Y_true_upper[j] == sum(weights_upper[i] * Xs_true_upper[i][j] for i in range(len(others))) + b_upper_value])
            touch_set_lower = set([true_objects[j] for j in range(len(Y_true_lower)) if Y_true_lower[j] == sum(weights_lower[i] * Xs_true_lower[i][j] for i in range(len(others))) + b_lower_value])

            touch_upper = len(touch_set_upper)
            touch_lower = len(touch_set_lower)

            # Create the hypothesis and conclusion objects for both upper and lower bounds.
            hypothesis = Hypothesis(hyp, true_object_set=true_objects)
            upper_conclusion = MultiLinearConclusion(target, "<=", weights_upper, others, b_upper_value)
            lower_conclusion = MultiLinearConclusion(target, ">=", weights_lower, others, b_lower_value)

            # Return the full conjecture object (not the conclusion directly).
            return MultiLinearConjecture(hypothesis, upper_conclusion, symbol, touch_upper, touch_set_upper), \
                MultiLinearConjecture(hypothesis, lower_conclusion, symbol, touch_lower, touch_set_lower)

def make_all_linear_conjectures(df, target, others, properties):
    # Create conjectures for every pair of invariants in 'others' combined with each property.
    upper_conjectures = []
    lower_conjectures = []
    seen_pairs = []
    for other1, other2 in combinations(others, 2):
        set_pair = set([other1, other2])
        if set_pair not in seen_pairs:
            seen_pairs.append(set_pair)
            for prop in properties:
                # Ensure that neither of the 'other' invariants equals the target.
                if other1 != target and other2 != target:
                    # Generate the conjecture for this combination of two invariants.
                    upper_conj, lower_conj = make_linear_conjectures(df, target, [other1, other2], hyp=prop)
                    upper_conjectures.append(upper_conj)
                    if lower_conj:
                        lower_conjectures.append(lower_conj)

    return upper_conjectures, lower_conjectures


In [3]:
def hazel_heuristic(conjectures, min_touch=0):
    # Remove duplicate conjectures.
    conjectures = list(set(conjectures))

    # Remove conjectures never attaining equality.
    conjectures = [conj for conj in conjectures if conj.touch > min_touch]

    # Sort the conjectures by touch number.
    conjectures.sort(key=lambda x: x.touch, reverse=True)

    # Return the sorted list of conjectures.
    return conjectures


def morgan_heuristic(conjectures):
    """
    Removes redundant conjectures from a list.

    A conjecture is considered redundant if another conjecture has the same conclusion
    and a more general hypothesis (i.e., its true_object_set is a superset of the redundant one).

    Parameters:
    conjectures (list of Conjecture): The list of conjectures to filter.

    Returns:
    list of Conjectures: A list with redundant conjectures removed.
    """
    new_conjectures = conjectures.copy()

    for conj_one in conjectures:
        for conj_two in new_conjectures.copy():  # Make a copy for safe removal
            # Avoid comparing the conjecture with itself
            if conj_one != conj_two:
                # Check if conclusions are the same and conj_one's hypothesis is more general
                if conj_one.conclusion == conj_two.conclusion and conj_one.hypothesis > conj_two.hypothesis:
                    new_conjectures.remove(conj_two)  # Remove the less general conjecture (conj_two)

    return new_conjectures


def weak_smokey(conjectures):
    # Start with the conjecture that has the highest touch number (first in the list).
    conj = conjectures[0]

    # Initialize the list of strong conjectures with the first conjecture.
    strong_conjectures = [conj]

    # Get the set of sharp graphs (i.e., graphs where the conjecture holds as equality) for the first conjecture.
    sharp_graphs = conj.sharps

    # Iterate over the remaining conjectures in the list.
    for conj in conjectures[1:]:
        if conj.is_equal():
            strong_conjectures.append(conj)
            sharp_graphs = sharp_graphs.union(conj.sharps)
        else:
            # Check if the current conjecture shares the same sharp graphs as any already selected strong conjecture.
            if any(conj.sharps.issuperset(known.sharps) for known in strong_conjectures):
                # If it does, add the current conjecture to the list of strong conjectures.
                strong_conjectures.append(conj)
                # Update the set of sharp graphs to include the newly discovered sharp graphs.
                sharp_graphs = sharp_graphs.union(conj.sharps)
            # Otherwise, check if the current conjecture introduces new sharp graphs (graphs where the conjecture holds).
            elif conj.sharps - sharp_graphs != set():
                # If new sharp graphs are found, add the conjecture to the list.
                strong_conjectures.append(conj)
                # Update the set of sharp graphs to include the newly discovered sharp graphs.
                sharp_graphs = sharp_graphs.union(conj.sharps)

    # Return the list of strong, non-redundant conjectures.
    return strong_conjectures


def strong_smokey(conjectures):
    # Start with the conjecture that has the highest touch number (first in the list).
    conj = conjectures[0]

    # Initialize the list of strong conjectures with the first conjecture.
    strong_conjectures = [conj]

    # Get the set of sharp graphs (i.e., graphs where the conjecture holds as equality) for the first conjecture.
    sharp_graphs = conj.sharps


    # Iterate over the remaining conjectures in the list.
    for conj in conjectures[1:]:
        if conj.is_equal():
            strong_conjectures.append(conj)
        else:
            # Check if the current conjecture set of sharp graphs is a superset of any already selected strong conjecture.
            if any(conj.sharps.issuperset(known.sharps) for known in strong_conjectures):
                # If it does, add the current conjecture to the list of strong conjectures.
                strong_conjectures.append(conj)
                sharp_graphs = sharp_graphs.union(conj.sharps)

    # Return the list of strong, non-redundant conjectures.
    return strong_conjectures


def filter_false_conjectures(conjectures, df):
    new_conjectures = []
    for conj in conjectures:
        if conj.false_graphs(df).empty:
            new_conjectures.append(conj)
    return new_conjectures


In [4]:
import pandas as pd

class Optimist:
    def __init__(self, graphs : list, invariants : dict, known_theorems=[]):
        """
        :param graphs: A list of initial graphs to start with.
        :param functions: A dictionary where keys are function names and values are the functions themselves.
        :param known_theorems: A list of known theorems to filter out redundant conjectures.
        """
        self.graphs = graphs
        self.invariants = invariants
        self.upper_conjectures = {}
        self.lower_conjectures = {}
        self.all_conjectures = {}
        self.known_theorems = known_theorems

    def build_knowledge(self):
        # Initialize an empty DataFrame
        rows = []
        for i, G in enumerate(self.graphs):
            row = {"name": f"G{i}"}
            for name, function in self.invariants.items():
                row[name] = function(G)
            rows.append(row)
        self.df = pd.DataFrame(rows)

    def conjecture(self, target, use_strong_smokey=False):
        numerical_columns = self.df.select_dtypes(include=['int64', 'float64']).columns.tolist()
        boolean_columns = self.df.select_dtypes(include=['bool']).columns.tolist()
        upper_conjectures, lower_conjectures = make_all_linear_conjectures(self.df, target, numerical_columns, boolean_columns)
        upper_conjectures = filter_false_conjectures(upper_conjectures, self.df)
        lower_conjectures = filter_false_conjectures(lower_conjectures, self.df)
        upper_conjectures, lower_conjectures = hazel_heuristic(upper_conjectures, min_touch=1), hazel_heuristic(lower_conjectures, min_touch=1)
        upper_conjectures, lower_conjectures = morgan_heuristic(upper_conjectures), morgan_heuristic(lower_conjectures)
        upper_conjectures = [conj for conj in upper_conjectures if conj not in self.known_theorems]
        lower_conjectures = [conj for conj in lower_conjectures if conj not in self.known_theorems]
        if use_strong_smokey:
            upper_conjectures, lower_conjectures = strong_smokey(upper_conjectures), strong_smokey(lower_conjectures)
        else:
            upper_conjectures, lower_conjectures = weak_smokey(upper_conjectures), weak_smokey(lower_conjectures)
        self.upper_conjectures[target] = upper_conjectures
        self.lower_conjectures[target] = lower_conjectures
        self.all_conjectures[target] = upper_conjectures + lower_conjectures
        return upper_conjectures, lower_conjectures

    def write_on_the_wall(self, target):
        if target not in self.upper_conjectures or target not in self.lower_conjectures:
            self.conjecture(target)
        for i, upper_conj in enumerate(self.upper_conjectures[target]):
            print(f"Conjecture {i+1}. {upper_conj}")
            print(f"With equality on {upper_conj.touch} graphs.\n")

        print()
        for i, lower_conj in enumerate(self.lower_conjectures[target]):
            print(f"Conjecture {i+1}. {lower_conj}")
            print(f"With equality on {lower_conj.touch} graphs.\n")

    def update_graph_knowledge(self, graph):
        self.graphs.append(graph)
        new_row = {"name": f"G{len(self.graphs)}"}
        for name, function in self.invariants.items():
            new_row[name] = function(graph)
        self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index=True)

    def update_known_theorems(self, theorem):
        self.known_theorems.append(theorem)

    def filter_known_upper_bounds(self, target, index):
        theorem = self.upper_conjectures[target][index - 1]
        self.known_theorems.append(theorem)
        self.conjecture(target)

    def filter_known_lower_bounds(self, target, index):
        theorem = self.lower_conjectures[target][index - 1]
        self.known_theorems.append(theorem)
        self.conjecture(target)


In [5]:
import networkx as nx
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, LpMinimize

# The neighborhood of a vertex v in G.
def neighborhood(G, v):
    return [n for n in G.neighbors(v)]

    # The closed neighborhood of a vertex v in G.
def closed_neighborhood(G, v):
    return [n for n in G.neighbors(v)] + [v]

# The order of the graph - the number of vertices.
def n(G):
    return G.number_of_nodes()

# The matching number of a graph - the cardinality of a maximum set of edges
# which do not share an endpoint.
def matching_number(G):
    prob = LpProblem("MaximumMatchingSet", LpMaximize)
    variables = {edge: LpVariable("x{}".format(i + 1), 0, 1, LpBinary) for i, edge in enumerate(G.edges())}

    # Set the maximum matching objective function
    prob += lpSum(variables)

    # Set constraints
    for node in G.nodes():
        incident_edges = [variables[edge] for edge in variables if node in edge]
        prob += sum(incident_edges) <= 1

    prob.solve()
    solution_set = {edge for edge in variables if variables[edge].value() == 1}
    return len(solution_set)

def domination_number(G):
    prob = LpProblem("MinDominatingSet", LpMinimize)
    variables = {node: LpVariable("x{}".format(i + 1), 0, 1, LpBinary) for i, node in enumerate(G.nodes())}

    # Set the total domination number objective function
    prob += lpSum([variables[n] for n in variables])

    # Set constraints
    for node in G.nodes():
        combination = [variables[n] for n in variables if n in closed_neighborhood(G, node)]
        prob += lpSum(combination) >= 1

    prob.solve()
    solution_set = {node for node in variables if variables[node].value() == 1}
    return len(solution_set)

def edge_domination_number(G):
    G = nx.line_graph(G)
    return domination_number(G)

def min_maximal_matching_number(G):
    return edge_domination_number(G)

def harmonic_index(G):
    return sum(2/(G.degree(v) + G.degree(u)) for v, u in G.edges())

# The independence number - the cardinality of a maximum set of pairwise nonadjacent
# vertices.
def independence_number(G):
    prob = LpProblem("MaximumIndependentSet", LpMaximize)
    variables = {node: LpVariable("x{}".format(i + 1), 0, 1, LpBinary) for i, node in enumerate(G.nodes())}

    # Set the domination number objective function
    prob += lpSum(variables)

    # Set constraints for independence
    for e in G.edges():
        prob += variables[e[0]] + variables[e[1]] <= 1

    prob.solve()
    solution_set = {node for node in variables if variables[node].value() == 1}
    return len(solution_set)

# The minimum vertex degree of the graph.
def minimum_degree(G):
    return min(G.degree(node) for node in G.nodes())

# The maximum vertex degree of the graph
def maximum_degree(G):
    return max(G.degree(node) for node in G.nodes())

# The maximum degree squared.
def maximum_degree_squared(G):
    return maximum_degree(G) ** 2

# Is the graph connected?
def connected_graph(G):
    return nx.is_connected(G)

# Is the graph a tree?
def tree(G):
    return nx.is_tree(G)

# Is the graph regular?
def connected_and_regular_graph(G):
    return nx.is_connected(G) and minimum_degree(G) == maximum_degree(G)

# Is the graph bipartite?
def connected_and_bipartite_graph(G):
    return nx.is_connected(G) and nx.is_bipartite(G)

def connected_and_regular_graph(G):
    return nx.is_connected(G) and minimum_degree(G) == maximum_degree(G)

In [6]:
graphs = [nx.complete_graph(2), nx.complete_graph(3), nx.path_graph(3)]

target = "independence_number"

invariants = {
        "a connected graph": connected_graph,
        "a connected and bipartite graph": connected_and_bipartite_graph,
        "a connected and regular graph": connected_and_regular_graph,
        "a tree": tree,
        "independence_number": independence_number,
        "order": n,
        "min_maximal_matching_number": min_maximal_matching_number,
        "matching_number": matching_number,
        "maximum_degree": maximum_degree,
        "minimum_degree": minimum_degree,
    }

optimist = Optimist(graphs, invariants)

In [7]:
optimist.build_knowledge()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/f8f0d0bf77734cc297e0a912619b76d5-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/f8f0d0bf77734cc297e0a912619b76d5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 15 RHS
At line 17 BOUNDS
At line 20 ENDATA
Problem MODEL has 1 rows, 2 columns and 2 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -1 to -1.79769e+308
Probing was tried 0 

In [8]:
optimist.df

,name,a connected graph,a connected and bipartite graph,a connected and regular graph,a tree,independence_number,order,min_maximal_matching_number,matching_number,maximum_degree,minimum_degree
0,G0,True,True,True,True,1,2,1,1,1,1
1,G1,True,False,True,False,1,3,1,1,2,2
2,G2,True,True,False,True,2,3,1,1,2,1


In [9]:
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/249ffa2141a148b6a037900d6ecce7d1-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/249ffa2141a148b6a037900d6ecce7d1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 70 RHS
At line 83 BOUNDS
At line 100 ENDATA
Problem MODEL has 12 rows, 10 columns and 40 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4 - 0.00 seconds
Cgl0004I processed model has 12 rows, 10 columns (4 integer (4 of which binary)) and 40 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -4
Cbc0038I Relaxing continuous give

([If G is a connected graph, then independence_number = order - minimum_degree.,
  If G is a connected and regular graph, then independence_number = 1/2 * min_maximal_matching_number + 1/2.,
  If G is a connected and bipartite graph, then independence_number = 3 * order -2 * maximum_degree - 3.,
  If G is a connected and regular graph, then independence_number = 1/2 * matching_number + 1/2.,
  If G is a connected and bipartite graph, then independence_number = order -3/2 * minimum_degree + 1/2.,
  If G is a connected and bipartite graph, then independence_number = order -3/2 * matching_number + 1/2.,
  If G is a connected graph, then independence_number <= order -3/2 * min_maximal_matching_number + 1/2.,
  If G is a connected and bipartite graph, then independence_number = order -3/2 * min_maximal_matching_number + 1/2.,
  If G is a connected graph, then independence_number <= -1/2 * min_maximal_matching_number + maximum_degree + 1/2.,
  If G is a connected graph, then independence_num

In [10]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected graph, then independence_number = order - minimum_degree
With equality on 3 graphs.

Conjecture 2. If G is a connected and regular graph, then independence_number = 1/2 * min_maximal_matching_number + 1/2
With equality on 2 graphs.

Conjecture 3. If G is a connected and bipartite graph, then independence_number = 3 * order -2 * maximum_degree - 3
With equality on 2 graphs.

Conjecture 4. If G is a connected and regular graph, then independence_number = 1/2 * matching_number + 1/2
With equality on 2 graphs.

Conjecture 5. If G is a connected and bipartite graph, then independence_number = order -3/2 * minimum_degree + 1/2
With equality on 2 graphs.

Conjecture 6. If G is a connected and bipartite graph, then independence_number = order -3/2 * matching_number + 1/2
With equality on 2 graphs.

Conjecture 7. If G is a connected graph, then independence_number <= order -3/2 * min_maximal_matching_number + 1/2
With equality on 2 graphs.

Conjecture 8. If G i

In [11]:
# Conjecture 1 is False in the upper bounds for a path graph large enough.
G_new = nx.path_graph(4)
optimist.update_graph_knowledge(G_new)
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/2f3dc2b008f14688952271d4e1b3d047-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/2f3dc2b008f14688952271d4e1b3d047-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 27 RHS
At line 31 BOUNDS
At line 36 ENDATA
Problem MODEL has 3 rows, 4 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -2 to -1.79769e+308
Probing was tried 0 

([If G is a connected and bipartite graph, then independence_number = maximum_degree.,
  If G is a connected graph, then independence_number <= 5/2 * min_maximal_matching_number - minimum_degree + 1/2.,
  If G is a tree, then independence_number = maximum_degree.,
  If G is a connected graph, then independence_number <= -1/2 * min_maximal_matching_number + maximum_degree + 1/2.,
  If G is a connected graph, then independence_number <= order - matching_number.,
  If G is a tree, then independence_number = -1/2 * min_maximal_matching_number + maximum_degree + 1/2.,
  If G is a connected and bipartite graph, then independence_number = order - matching_number.,
  If G is a connected and bipartite graph, then independence_number = -1/2 * min_maximal_matching_number + maximum_degree + 1/2.,
  If G is a connected graph, then independence_number <= maximum_degree.,
  If G is a tree, then independence_number = maximum_degree -1/2 * minimum_degree + 1/2.,
  If G is a tree, then independence_numb

In [12]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected and bipartite graph, then independence_number = maximum_degree
With equality on 3 graphs.

Conjecture 2. If G is a connected graph, then independence_number <= 5/2 * min_maximal_matching_number - minimum_degree + 1/2
With equality on 3 graphs.

Conjecture 3. If G is a tree, then independence_number = maximum_degree
With equality on 3 graphs.

Conjecture 4. If G is a connected graph, then independence_number <= -1/2 * min_maximal_matching_number + maximum_degree + 1/2
With equality on 3 graphs.

Conjecture 5. If G is a connected graph, then independence_number <= order - matching_number
With equality on 3 graphs.

Conjecture 6. If G is a tree, then independence_number = -1/2 * min_maximal_matching_number + maximum_degree + 1/2
With equality on 3 graphs.

Conjecture 7. If G is a connected and bipartite graph, then independence_number = order - matching_number
With equality on 3 graphs.

Conjecture 8. If G is a connected and bipartite graph, then independ

In [13]:
# Conjecture 3 is False in the upper bounds for complete bipartite graphs.
G_new = nx.complete_bipartite_graph(3, 3)
optimist.update_graph_knowledge(G_new)
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/234a6cb884e148478e3c9ae6f0721684-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/234a6cb884e148478e3c9ae6f0721684-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 51 RHS
At line 61 BOUNDS
At line 68 ENDATA
Problem MODEL has 9 rows, 6 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3 - 0.00 seconds
Cgl0004I processed model has 9 rows, 6 columns (6 integer (6 of which binary)) and 18 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution fo

([If G is a connected and bipartite graph, then independence_number = maximum_degree.,
  If G is a connected graph, then independence_number <= order - matching_number.,
  If G is a connected graph, then independence_number <= order - minimum_degree.,
  If G is a connected and bipartite graph, then independence_number = order - matching_number.,
  If G is a connected graph, then independence_number <= maximum_degree.,
  If G is a connected and regular graph, then independence_number = order - maximum_degree.,
  If G is a connected and regular graph, then independence_number = min_maximal_matching_number.,
  If G is a connected and regular graph, then independence_number = matching_number.,
  If G is a connected and regular graph, then independence_number = 4 * min_maximal_matching_number -3 * matching_number.,
  If G is a tree, then independence_number = -1/2 * min_maximal_matching_number + maximum_degree + 1/2.,
  If G is a tree, then independence_number = maximum_degree -1/2 * minimu

In [14]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected and bipartite graph, then independence_number = maximum_degree
With equality on 4 graphs.

Conjecture 2. If G is a connected graph, then independence_number <= order - matching_number
With equality on 4 graphs.

Conjecture 3. If G is a connected graph, then independence_number <= order - minimum_degree
With equality on 4 graphs.

Conjecture 4. If G is a connected and bipartite graph, then independence_number = order - matching_number
With equality on 4 graphs.

Conjecture 5. If G is a connected graph, then independence_number <= maximum_degree
With equality on 4 graphs.

Conjecture 6. If G is a connected and regular graph, then independence_number = order - maximum_degree
With equality on 3 graphs.

Conjecture 7. If G is a connected and regular graph, then independence_number = min_maximal_matching_number
With equality on 3 graphs.

Conjecture 8. If G is a connected and regular graph, then independence_number = matching_number
With equality on 3 graphs

In [ ]:
# Several conjectured equalities are False for cycle graphs with large enough order.
G_new = nx.cycle_graph(6)
optimist.update_graph_knowledge(G_new)
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/7b03c59a0fb74950a1137920a3398d91-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/7b03c59a0fb74950a1137920a3398d91-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 42 RHS
At line 49 BOUNDS
At line 56 ENDATA
Problem MODEL has 6 rows, 6 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3 - 0.00 seconds
Cgl0004I processed model has 6 rows, 6 columns (6 integer (6 of which binary)) and 12 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution fo

([If G is a connected graph, then independence_number <= order - matching_number.,
  If G is a connected and bipartite graph, then independence_number = order - matching_number.,
  If G is a connected and regular graph, then independence_number = matching_number.,
  If G is a tree, then independence_number = maximum_degree.,
  If G is a tree, then independence_number = -1/2 * min_maximal_matching_number + maximum_degree + 1/2.,
  If G is a tree, then independence_number = maximum_degree -1/2 * minimum_degree + 1/2.],
 [If G is a connected graph, then independence_number >= matching_number.,
  If G is a connected and bipartite graph, then independence_number >= maximum_degree.])

In [16]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected graph, then independence_number <= order - matching_number
With equality on 5 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number = order - matching_number
With equality on 5 graphs.

Conjecture 3. If G is a connected and regular graph, then independence_number = matching_number
With equality on 4 graphs.

Conjecture 4. If G is a tree, then independence_number = maximum_degree
With equality on 3 graphs.

Conjecture 5. If G is a tree, then independence_number = -1/2 * min_maximal_matching_number + maximum_degree + 1/2
With equality on 3 graphs.

Conjecture 6. If G is a tree, then independence_number = maximum_degree -1/2 * minimum_degree + 1/2
With equality on 3 graphs.


Conjecture 1. If G is a connected graph, then independence_number >= matching_number
With equality on 5 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number >= maximum_degree
With equality on 4 graphs.



In [17]:
# Conjecture 3 is false for the Petersen graph
G_new = nx.petersen_graph()
optimist.update_graph_knowledge(G_new)
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/8511f75167f14f6aaa6e70805e427382-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/8511f75167f14f6aaa6e70805e427382-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 20 COLUMNS
At line 81 RHS
At line 97 BOUNDS
At line 108 ENDATA
Problem MODEL has 15 rows, 10 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5 - 0.00 seconds
Cgl0004I processed model has 15 rows, 10 columns (10 integer (10 of which binary)) and 30 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 10 integers unsatisfied sum - 5
Cbc0038I Sol

([If G is a connected graph, then independence_number <= order - matching_number.,
  If G is a connected and bipartite graph, then independence_number = order - matching_number.,
  If G is a connected graph, then independence_number <= order - minimum_degree.,
  If G is a connected graph, then independence_number <= min_maximal_matching_number + 1.,
  If G is a connected graph, then independence_number <= minimum_degree + 1.,
  If G is a tree, then independence_number = maximum_degree.,
  If G is a tree, then independence_number = -1/2 * min_maximal_matching_number + maximum_degree + 1/2.,
  If G is a tree, then independence_number = maximum_degree -1/2 * minimum_degree + 1/2.],
 [If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2.,
  If G is a connected and bipartite graph, then independence_number >= 1/2 * order.,
  If G is a connected and bipartite graph, then independence_number >= matching_number.,
  If G is a connected and bipartite gra

In [18]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected graph, then independence_number <= order - matching_number
With equality on 5 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number = order - matching_number
With equality on 5 graphs.

Conjecture 3. If G is a connected graph, then independence_number <= order - minimum_degree
With equality on 4 graphs.

Conjecture 4. If G is a connected graph, then independence_number <= min_maximal_matching_number + 1
With equality on 4 graphs.

Conjecture 5. If G is a connected graph, then independence_number <= minimum_degree + 1
With equality on 4 graphs.

Conjecture 6. If G is a tree, then independence_number = maximum_degree
With equality on 3 graphs.

Conjecture 7. If G is a tree, then independence_number = -1/2 * min_maximal_matching_number + maximum_degree + 1/2
With equality on 3 graphs.

Conjecture 8. If G is a tree, then independence_number = maximum_degree -1/2 * minimum_degree + 1/2
With equality on 3 graphs.


Conjectu

In [ ]:
# Conjecture 7 is false for long enough paths.
G_new = nx.path_graph(5)
optimist.update_graph_knowledge(G_new)
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/a6d9641a310646eeb42b016b0cd009d7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/a6d9641a310646eeb42b016b0cd009d7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 33 RHS
At line 38 BOUNDS
At line 44 ENDATA
Problem MODEL has 4 rows, 5 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -3 to -1.79769e+308
Probing was tried 0 

([If G is a connected graph, then independence_number <= order - matching_number.,
  If G is a connected and bipartite graph, then independence_number = order - matching_number.,
  If G is a connected graph, then independence_number <= min_maximal_matching_number + 1.,
  If G is a tree, then independence_number = min_maximal_matching_number + maximum_degree - 1.,
  If G is a connected graph, then independence_number <= order - minimum_degree.],
 [If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2.,
  If G is a connected and bipartite graph, then independence_number >= 1/2 * order.,
  If G is a connected and bipartite graph, then independence_number >= matching_number.,
  If G is a connected and bipartite graph, then independence_number >= maximum_degree.,
  If G is a tree, then independence_number >= min_maximal_matching_number + matching_number - 1.,
  If G is a tree, then independence_number >= 1/2 * order + 1/2 * min_maximal_matching_numbe

In [20]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected graph, then independence_number <= order - matching_number
With equality on 6 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number = order - matching_number
With equality on 6 graphs.

Conjecture 3. If G is a connected graph, then independence_number <= min_maximal_matching_number + 1
With equality on 5 graphs.

Conjecture 4. If G is a tree, then independence_number = min_maximal_matching_number + maximum_degree - 1
With equality on 4 graphs.

Conjecture 5. If G is a connected graph, then independence_number <= order - minimum_degree
With equality on 4 graphs.


Conjecture 1. If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2
With equality on 4 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number >= 1/2 * order
With equality on 4 graphs.

Conjecture 3. If G is a connected and bipartite graph, then independence_number >= matching_number
Wi

In [21]:
# Conjecture 3 is false for large enough star graphs
G_new = nx.star_graph(4)
optimist.update_graph_knowledge(G_new)
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/608aa80fa5d64b3394e41f544be64fa4-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/608aa80fa5d64b3394e41f544be64fa4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 33 RHS
At line 38 BOUNDS
At line 44 ENDATA
Problem MODEL has 4 rows, 5 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -4 to -1.79769e+308
Probing was tried 0 

([If G is a connected graph, then independence_number <= order - matching_number.,
  If G is a connected and bipartite graph, then independence_number = order - matching_number.,
  If G is a tree, then independence_number = min_maximal_matching_number + maximum_degree - 1.,
  If G is a connected graph, then independence_number <= order - minimum_degree.,
  If G is a connected graph, then independence_number <= maximum_degree + 1.],
 [If G is a connected and bipartite graph, then independence_number >= maximum_degree.,
  If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2.,
  If G is a connected and bipartite graph, then independence_number >= 1/2 * order.,
  If G is a connected and bipartite graph, then independence_number >= matching_number.,
  If G is a tree, then independence_number >= min_maximal_matching_number + matching_number - 1.,
  If G is a tree, then independence_number >= 1/2 * order + 1/2 * min_maximal_matching_number - 1/2.])

In [22]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected graph, then independence_number <= order - matching_number
With equality on 7 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number = order - matching_number
With equality on 7 graphs.

Conjecture 3. If G is a tree, then independence_number = min_maximal_matching_number + maximum_degree - 1
With equality on 5 graphs.

Conjecture 4. If G is a connected graph, then independence_number <= order - minimum_degree
With equality on 5 graphs.

Conjecture 5. If G is a connected graph, then independence_number <= maximum_degree + 1
With equality on 3 graphs.


Conjecture 1. If G is a connected and bipartite graph, then independence_number >= maximum_degree
With equality on 5 graphs.

Conjecture 2. If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2
With equality on 4 graphs.

Conjecture 3. If G is a connected and bipartite graph, then independence_number >= 1/2 * order
With equality on

In [23]:
# Conjecture 4 is false for stars with a leaf attached to each leaf
G_new = nx.Graph()
G_new.add_edges_from([(0, 1), (0, 2), (0, 3), (0, 4), (1, 5), (2, 6), (3, 7), (4, 8)])
optimist.update_graph_knowledge(G_new)
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/dfa0dbd13b8b4780838edcfa577a28c8-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/dfa0dbd13b8b4780838edcfa577a28c8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 57 RHS
At line 66 BOUNDS
At line 76 ENDATA
Problem MODEL has 8 rows, 9 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -5 to -1.79769e+308
Probing was tried 

([If G is a connected graph, then independence_number <= order - matching_number.,
  If G is a connected and bipartite graph, then independence_number = order - matching_number.,
  If G is a connected graph, then independence_number <= order - minimum_degree.,
  If G is a connected graph, then independence_number <= maximum_degree + 1.],
 [If G is a connected graph, then independence_number >= 1/4 * order + 5/4 * maximum_degree - 9/4.,
  If G is a connected and bipartite graph, then independence_number >= maximum_degree.,
  If G is a connected and bipartite graph, then independence_number >= 1/2 * order.,
  If G is a connected and bipartite graph, then independence_number >= matching_number.])

In [24]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected graph, then independence_number <= order - matching_number
With equality on 8 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number = order - matching_number
With equality on 8 graphs.

Conjecture 3. If G is a connected graph, then independence_number <= order - minimum_degree
With equality on 5 graphs.

Conjecture 4. If G is a connected graph, then independence_number <= maximum_degree + 1
With equality on 4 graphs.


Conjecture 1. If G is a connected graph, then independence_number >= 1/4 * order + 5/4 * maximum_degree - 9/4
With equality on 5 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number >= maximum_degree
With equality on 5 graphs.

Conjecture 3. If G is a connected and bipartite graph, then independence_number >= 1/2 * order
With equality on 4 graphs.

Conjecture 4. If G is a connected and bipartite graph, then independence_number >= matching_number
With equality on 4 gra

In [25]:
# Conjecture 4 (upper) is false for large enough paths
G_new = nx.path_graph(10)
optimist.update_graph_knowledge(G_new)
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/57d0d7eb1e9848719f4cde6aa0861642-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/57d0d7eb1e9848719f4cde6aa0861642-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 63 RHS
At line 73 BOUNDS
At line 84 ENDATA
Problem MODEL has 9 rows, 10 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -5 to -1.79769e+308
Probing was tried

([If G is a connected graph, then independence_number <= order - matching_number.,
  If G is a connected and bipartite graph, then independence_number = order - matching_number.,
  If G is a connected graph, then independence_number <= order - minimum_degree.,
  If G is a connected graph, then independence_number <= 3 * maximum_degree -2 * minimum_degree + 1.],
 [If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2.,
  If G is a connected graph, then independence_number >= 1/4 * order + 5/4 * maximum_degree - 9/4.,
  If G is a connected and bipartite graph, then independence_number >= 1/2 * order.,
  If G is a connected and bipartite graph, then independence_number >= matching_number.,
  If G is a connected and bipartite graph, then independence_number >= maximum_degree.])

In [26]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected graph, then independence_number <= order - matching_number
With equality on 9 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number = order - matching_number
With equality on 9 graphs.

Conjecture 3. If G is a connected graph, then independence_number <= order - minimum_degree
With equality on 5 graphs.

Conjecture 4. If G is a connected graph, then independence_number <= 3 * maximum_degree -2 * minimum_degree + 1
With equality on 3 graphs.


Conjecture 1. If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2
With equality on 5 graphs.

Conjecture 2. If G is a connected graph, then independence_number >= 1/4 * order + 5/4 * maximum_degree - 9/4
With equality on 5 graphs.

Conjecture 3. If G is a connected and bipartite graph, then independence_number >= 1/2 * order
With equality on 5 graphs.

Conjecture 4. If G is a connected and bipartite graph, then independence_number >= mat

In [27]:
# Conjecture 4 (upper is false for many regular graohs)
G_new = nx.cycle_graph(10)
optimist.update_graph_knowledge(G_new)
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/205b77c67a2b43d3b01ba2c5fee84bd7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/205b77c67a2b43d3b01ba2c5fee84bd7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 66 RHS
At line 77 BOUNDS
At line 88 ENDATA
Problem MODEL has 10 rows, 10 columns and 20 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5 - 0.00 seconds
Cgl0004I processed model has 10 rows, 10 columns (10 integer (10 of which binary)) and 20 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solut

([If G is a connected graph, then independence_number <= order - matching_number.,
  If G is a connected and bipartite graph, then independence_number = order - matching_number.,
  If G is a connected graph, then independence_number <= order - minimum_degree.],
 [If G is a connected and bipartite graph, then independence_number >= 1/2 * order.,
  If G is a connected and bipartite graph, then independence_number >= matching_number.,
  If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2.,
  If G is a connected graph, then independence_number >= 1/4 * order + 5/4 * maximum_degree - 9/4.,
  If G is a connected and bipartite graph, then independence_number >= maximum_degree.])

In [28]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected graph, then independence_number <= order - matching_number
With equality on 10 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number = order - matching_number
With equality on 10 graphs.

Conjecture 3. If G is a connected graph, then independence_number <= order - minimum_degree
With equality on 5 graphs.


Conjecture 1. If G is a connected and bipartite graph, then independence_number >= 1/2 * order
With equality on 6 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number >= matching_number
With equality on 6 graphs.

Conjecture 3. If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2
With equality on 5 graphs.

Conjecture 4. If G is a connected graph, then independence_number >= 1/4 * order + 5/4 * maximum_degree - 9/4
With equality on 5 graphs.

Conjecture 5. If G is a connected and bipartite graph, then independence_number >= maximum_degree

In [29]:
upper_conjectures = optimist.upper_conjectures["independence_number"]
lower_conjectures = optimist.lower_conjectures["independence_number"]

for i, conj in enumerate(upper_conjectures):
    print(f"Conjecture {i+1}. {conj}")
    print(conj.sharps)
    print()

for i, conj in enumerate(lower_conjectures):
    print(f"Conjecture {i+1}. {conj}")
    print(conj.sharps)
    print()

Conjecture 1. If G is a connected graph, then independence_number <= order - matching_number
{'G6', 'G9', 'G4', 'G10', 'G8', 'G2', 'G0', 'G5', 'G12', 'G11'}

Conjecture 2. If G is a connected and bipartite graph, then independence_number = order - matching_number
{'G6', 'G9', 'G4', 'G10', 'G8', 'G2', 'G0', 'G5', 'G12', 'G11'}

Conjecture 3. If G is a connected graph, then independence_number <= order - minimum_degree
{'G9', 'G1', 'G0', 'G5', 'G2'}

Conjecture 1. If G is a connected and bipartite graph, then independence_number >= 1/2 * order
{'G6', 'G4', 'G0', 'G5', 'G11', 'G12'}

Conjecture 2. If G is a connected and bipartite graph, then independence_number >= matching_number
{'G6', 'G4', 'G0', 'G5', 'G11', 'G12'}

Conjecture 3. If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2
{'G7', 'G1', 'G4', 'G0', 'G11'}

Conjecture 4. If G is a connected graph, then independence_number >= 1/4 * order + 5/4 * maximum_degree - 9/4
{'G9', 'G7', 'G1', 'G

In [ ]:
# The first three upper bound conjectures are known theorems.
theorems = [upper_conjectures[0], upper_conjectures[1], upper_conjectures[2]]
optimist.update_known_theorems(theorems)
optimist.conjecture("independence_number")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/The-Optimist/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/cfd129476ba54037bb1792634985d530-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/cfd129476ba54037bb1792634985d530-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 65 COLUMNS
At line 326 RHS
At line 387 BOUNDS
At line 420 ENDATA
Problem MODEL has 60 rows, 26 columns and 200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 19.989 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 11 strengthened rows, 0 substitutions
Cgl0004I processed model has 60 rows, 26 columns (20 integer (20 of which binary)) and 200 elements
Cbc0038I Initial state - 11 inte

([If G is a connected graph, then independence_number <= order - matching_number.,
  If G is a connected and bipartite graph, then independence_number = order - matching_number.,
  If G is a connected graph, then independence_number <= order - minimum_degree.],
 [If G is a connected and bipartite graph, then independence_number >= 1/2 * order.,
  If G is a connected and bipartite graph, then independence_number >= matching_number.,
  If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2.,
  If G is a connected graph, then independence_number >= 1/4 * order + 5/4 * maximum_degree - 9/4.,
  If G is a connected and bipartite graph, then independence_number >= maximum_degree.])

In [31]:
optimist.write_on_the_wall("independence_number")

Conjecture 1. If G is a connected graph, then independence_number <= order - matching_number
With equality on 10 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number = order - matching_number
With equality on 10 graphs.

Conjecture 3. If G is a connected graph, then independence_number <= order - minimum_degree
With equality on 5 graphs.


Conjecture 1. If G is a connected and bipartite graph, then independence_number >= 1/2 * order
With equality on 6 graphs.

Conjecture 2. If G is a connected and bipartite graph, then independence_number >= matching_number
With equality on 6 graphs.

Conjecture 3. If G is a connected graph, then independence_number >= 1/2 * order -1/2 * minimum_degree + 1/2
With equality on 5 graphs.

Conjecture 4. If G is a connected graph, then independence_number >= 1/4 * order + 5/4 * maximum_degree - 9/4
With equality on 5 graphs.

Conjecture 5. If G is a connected and bipartite graph, then independence_number >= maximum_degree